In [ ]:
import pandas as pd
data_after_outlier1 = pd.read_csv("C:\\Users\\ganch\\Desktop\\Independent Study\\python_twitter_collection_sample\\binned_data.csv")

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(1,3,1)
plt.hist(data_after_outlier.bin_class_Retweet)
plt.subplot(1,3,3)
plt.hist(data_after_outlier.bin_class_Likes)

# Downsampling Retweets

In [ ]:
count_class_1, count_class_2,count_class_3,count_class_4 = data_after_outlier1.bin_class_Retweet.value_counts()

# Divide by class
df_class_1 = data_after_outlier1[data_after_outlier1['bin_class_Retweet'] == 1]
df_class_2 = data_after_outlier1[data_after_outlier1['bin_class_Retweet'] == 2]
df_class_3 = data_after_outlier1[data_after_outlier1['bin_class_Retweet'] == 3]
df_class_4 = data_after_outlier1[data_after_outlier1['bin_class_Retweet'] == 4]

df_class_1_under = df_class_1.sample(count_class_4,replace = True)
df_class_2_under = df_class_2.sample(count_class_4,replace = True)
df_class_3_under = df_class_3.sample(count_class_4,replace = True)
df_class_4_under = df_class_4.sample(count_class_4,replace = True)
df_test_under = pd.concat([df_class_1_under, df_class_2_under,df_class_3_under,df_class_4_under], axis=0)

print('Random under-sampling:')
print(df_test_under.bin_class_Retweet.value_counts())

df_test_under.bin_class_Retweet.value_counts().plot(kind='bar', title='Count (bin_class_Retweet)')

# Downsampling Likes

In [ ]:
count_class_1, count_class_2,count_class_3,count_class_4 = data_after_outlier1.bin_class_Likes.value_counts()

# Divide by class
df_class_1 = data_after_outlier1[data_after_outlier1['bin_class_Likes'] == 1]
df_class_2 = data_after_outlier1[data_after_outlier1['bin_class_Likes'] == 2]
df_class_3 = data_after_outlier1[data_after_outlier1['bin_class_Likes'] == 3]
df_class_4 = data_after_outlier1[data_after_outlier1['bin_class_Likes'] == 4]

df_class_1_under = df_class_1.sample(count_class_4,replace = True)
df_class_2_under = df_class_2.sample(count_class_4,replace = True)
df_class_3_under = df_class_3.sample(count_class_4,replace = True)
df_class_4_under = df_class_4.sample(count_class_4,replace = True)
df_test_under_Likes = pd.concat([df_class_1_under, df_class_2_under,df_class_3_under,df_class_4_under], axis=0)

print('Random under-sampling:')
print(df_test_under_Likes.bin_class_Likes.value_counts())

df_test_under_Likes.bin_class_Likes.value_counts().plot(kind='bar', title='Count (bin_class_Likes)')

# Data Partitioning

In [ ]:
from sklearn.model_selection import train_test_split
X = df_test_under_Likes[["clean_text"]]
y = df_test_under_Likes['bin_class_Likes']
#y = df_test_under_Likes['bin_class_Retweet']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report,accuracy_score

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression())
])

grid_search = GridSearchCV(pipeline,param_grid = {'clf__solver': ('newton-cg', 'lbfgs', 'liblinear', 'saga')}, cv=5,
                               n_jobs=-1, verbose=1)

print("Performing grid search...")
#if type() is str:
#       tweet = tweet.lower()
grid_search.fit(list(X_train.clean_text), list(y_train))

print()

print("Best score: %0.3f" % grid_search.best_score_)
y_true, y_pred = y_test, grid_search.predict(list(X_test.clean_text))
accuracy_score(y_true,y_pred)

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC())
])

grid_search = GridSearchCV(pipeline,param_grid = {'clf__kernel':['linear', 'poly', 'rbf', 'sigmoid'],
            'clf__degree':[1, 2, 3, 4],
            'clf__C':[0.5, 1, 2]}, cv=5,
                               n_jobs=-1, verbose=1)

print("Performing grid search...")

grid_search.fit(list(X_train.clean_text), list(y_train))

print()

print("Best score: %0.3f" % grid_search.best_score_)
y_true, y_pred = y_test, grid_search.predict(list(X_test.clean_text))
accuracy_score(y_true,y_pred)

# Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

grid_search = GridSearchCV(pipeline,param_grid = {'clf__alpha': [0, 0.5, 1],
                'clf__fit_prior': [True, False]}, cv=5,
                               n_jobs=-1, verbose=1)

print("Performing grid search...")

grid_search.fit(list(X_train.clean_text), list(y_train))

print()

print("Best score: %0.3f" % grid_search.best_score_)
y_true, y_pred = y_test, grid_search.predict(list(X_test.clean_text))
accuracy_score(y_true,y_pred)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf'a, RandomForestClassifier())
])

grid_search = GridSearchCV(pipeline,param_grid = {'clf__max_features': [None],
                'clf__n_estimators': [100],'clf__min_samples_leaf' : [5]}, cv=5,
                               n_jobs=-1, verbose=1)

print("Performing grid search...")

grid_search.fit(list(X_train.clean_text), list(y_train))

print()

print("Best score: %0.3f" % grid_search.best_score_)
y_true, y_pred = y_test, grid_search.predict(list(X_test.clean_text))
accuracy_score(y_true,y_pred)

# Neural Networks

In [ ]:
import sklearn
a = df_test_under['bin_class_Retweet']
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(a)+1))
b = label_binarizer.transform(a)

# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer

X = df_test_under["clean_text"].values
vectorizer = TfidfVectorizer(
                             use_idf = True,
                            stop_words='english',
                            max_features=25000)
X = vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, b, test_size=0.30, random_state=42)

model1 = Sequential()
model1.add(Dense(512 ,input_dim=25000, activation='relu'))
model1.add(Dense(256, activation='sigmoid'))
model1.add(Dropout(0.3))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(5, activation='softmax'))
# Compile model

#use for narratives
sgd=optimizers.Adamax(decay=0.01)
##use for issues
# sgd=optimizers.SGD(lr=0.01, decay=0.001)

model1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

history=model1.fit(X_train, y_train,batch_size=512, epochs=100, validation_data=(X_test, y_test), verbose=2)